In [85]:
import pandas as pd

import geopandas as gpd

In [86]:
df_addresses = pd.read_csv('combined data\PCODE_2025_NUTS-2024_v1.0.csv')
df_loaction = pd.read_csv("combined data\\nuts_location.csv")
df_yild = pd.read_csv('combined data\crop_data_bd01802f-1adb-4883-abae-a2427e685d0f_2025.v01_39.csv', delimiter=";")

df_addresses

<>:1: SyntaxWarning: invalid escape sequence '\P'
<>:3: SyntaxWarning: invalid escape sequence '\c'
<>:1: SyntaxWarning: invalid escape sequence '\P'
<>:3: SyntaxWarning: invalid escape sequence '\c'
C:\Users\artur\AppData\Local\Temp\ipykernel_21216\393609053.py:1: SyntaxWarning: invalid escape sequence '\P'
  df_addresses = pd.read_csv('combined data\PCODE_2025_NUTS-2024_v1.0.csv')
C:\Users\artur\AppData\Local\Temp\ipykernel_21216\393609053.py:3: SyntaxWarning: invalid escape sequence '\c'
  df_yild = pd.read_csv('combined data\crop_data_bd01802f-1adb-4883-abae-a2427e685d0f_2025.v01_39.csv', delimiter=";")


,CODE,NUTS3
0,'5621 CX','NL414'
1,'6372 XR','NL423'
2,'9905 RM','NL112'
3,'5953 EE','NL421'
4,'5864 DB','NL421'
...,...,...
830028,'72186','LT027'
830029,'94148','LT023'
830030,'50464','LT022'
830031,'27115','LT011'


In [87]:
nuts_centroids = gpd.read_file('combined data\\nuts_lon_lat.geojson')
nuts_centroids['LONGITUDE'] = nuts_centroids.geometry.x
nuts_centroids['LATITUDE'] = nuts_centroids.geometry.y
nuts_coords = nuts_centroids[['NUTS_ID', 'LATITUDE', 'LONGITUDE']]
nuts_coords

,NUTS_ID,LATITUDE,LONGITUDE
0,DEA28,50.538151,6.641019
1,DEA29,51.052465,6.171671
2,DEA2A,51.021514,7.522380
3,DEA2B,51.022015,7.191688
4,DEA2C,50.772640,7.262610
...,...,...,...
2011,UKN12,55.061808,-6.594952
2012,UKN13,54.694314,-6.200551
2013,UKN14,54.505059,-6.040986
2014,UKN15,54.888586,-6.179498


In [88]:
df_loaction['NUTS_ID'].iloc[1]

'UKL18'

In [89]:

df_yild = df_yild.rename(columns={'IDREGION': 'NUTS_ID'})
print(len(df_yild["NUTS_ID"].unique()))
df_yild["NUTS_ID"].iloc[1]

901


'AT11'

In [90]:
avg_df = df_yild.groupby(['NUTS_ID', 'CROP_NAME', 'VARIABLE']).agg({
    'VALUE': 'mean',
    'YEAR': ['min', 'max', 'count'],  
    'UNIT': 'first',  # Keep the unit
    'SOURCE': 'first'  # Keep the source
}).reset_index()

# Flatten column names
avg_df.columns = ['NUTS_ID', 'CROP_NAME', 'VARIABLE', 'AVG_VALUE', 
                  'YEAR_MIN', 'YEAR_MAX', 'YEAR_COUNT', 'UNIT', 'SOURCE']

# Save to CSV
avg_df.to_csv('average_by_region.csv', index=False)

avg_df_yield = avg_df[avg_df['VARIABLE'] == 'Area']

print(f"Aggregated rows: {len(avg_df_yield)}")
print(len(avg_df["NUTS_ID"].unique()))
avg_df_yield.head(10)

Aggregated rows: 901
901


,NUTS_ID,CROP_NAME,VARIABLE,AVG_VALUE,YEAR_MIN,YEAR_MAX,YEAR_COUNT,UNIT,SOURCE
0,AT11,Total wheat,Area,44493.283529,2006,2022,17,ha,NSI
3,AT12,Total wheat,Area,179513.844118,2006,2022,17,ha,NSI
6,AT13,Total wheat,Area,1412.866471,2006,2022,17,ha,NSI
9,AT21,Total wheat,Area,2996.106875,2007,2022,16,ha,NSI
12,AT22,Total wheat,Area,6873.790588,2006,2022,17,ha,NSI
15,AT31,Total wheat,Area,48374.446471,2006,2022,17,ha,NSI
18,AT32,Total wheat,Area,254.060000,2008,2022,12,ha,NSI
21,AT33,Total wheat,Area,121.320625,2007,2022,16,ha,NSI
24,AT34,Total wheat,Area,9.270000,2008,2022,13,ha,NSI
27,BE10,Total wheat,Area,346.000000,2006,2016,10,ha,Mixed


In [91]:
df_yild.head(17).to_csv('example_of_yild_data.csv',index=False)

In [92]:
df_addresses['NUTS3'].iloc[1]

"'NL423'"

In [93]:
df_addresses['NUTS3'] = (
    df_addresses['NUTS3'].astype(str)
    .str.strip()
    .str.replace(r'^[\'"]|[\'"]$', '', regex=True))

In [94]:
df_addresses['NUTS3'].iloc[1]

'NL423'

In [ ]:
df_filtered = df_loaction[df_loaction['NUTS_ID'].isin(df_yild['NUTS_ID'])]
df_filtered

KeyError: 'IDREGION'

In [ ]:
df_regions = df_filtered[['NAME_LATN','NUTS_ID']]
df_regions

,NAME_LATN,NUTS_ID
86,Göttingen,DE91C
87,Cochem-Zell,DEB1C
88,Rhein-Hunsrück-Kreis,DEB1D
89,Kainuu,FI1D8
90,Pohjois-Pohjanmaa,FI1D9
...,...,...
2004,Borsod-Abaúj-Zemplén,HU311
2007,Málaga,ES617
2008,Erlangen-Höchstadt,DE257
2011,Fürstenfeldbruck,DE21C


In [ ]:
df_with_coords = df_regions.merge(nuts_coords, on='NUTS_ID', how='left')
df_with_coords

,NAME_LATN,NUTS_ID,LATITUDE,LONGITUDE
0,Göttingen,DE91C,51.498627,9.927105
1,Cochem-Zell,DEB1C,50.131592,7.164867
2,Rhein-Hunsrück-Kreis,DEB1D,50.041526,7.501010
3,Kainuu,FI1D8,64.499445,28.719371
4,Pohjois-Pohjanmaa,FI1D9,64.939882,26.622719
...,...,...,...,...
896,Borsod-Abaúj-Zemplén,HU311,48.205536,20.976582
897,Málaga,ES617,36.822229,-4.713850
898,Erlangen-Höchstadt,DE257,49.655389,10.967361
899,Fürstenfeldbruck,DE21C,48.181335,11.213405


In [ ]:
avg_df_yield_with_cords = avg_df_yield.merge(nuts_coords, on='NUTS_ID', how='left')
avg_df_yield_with_cords

,NUTS_ID,CROP_NAME,VARIABLE,AVG_VALUE,YEAR_MIN,YEAR_MAX,YEAR_COUNT,UNIT,SOURCE,LATITUDE,LONGITUDE
0,AT11,Total wheat,Yield,4.568235,2006,2022,17,t/ha,NSI,47.236076,16.287376
1,AT12,Total wheat,Yield,5.252353,2006,2022,17,t/ha,NSI,48.258880,15.760103
2,AT13,Total wheat,Yield,4.713529,2006,2022,17,t/ha,NSI,48.202775,16.393215
3,AT21,Total wheat,Yield,5.666875,2007,2022,16,t/ha,NSI,46.716091,14.462861
4,AT22,Total wheat,Yield,6.187647,2006,2022,17,t/ha,NSI,47.181814,15.524707
...,...,...,...,...,...,...,...,...,...,...,...
511,SK023,Total wheat,Yield,5.246250,2007,2022,16,t/ha,NSI,48.124811,18.320786
512,SK031,Total wheat,Yield,4.439286,2007,2022,14,t/ha,NSI,49.177208,19.168347
513,SK032,Total wheat,Yield,3.868571,2007,2022,14,t/ha,NSI,48.502294,19.512763
514,SK041,Total wheat,Yield,3.783333,2007,2022,15,t/ha,NSI,49.184233,21.225385


In [ ]:
avg_df_yield_with_cords.to_csv('avg_yield_cords.csv',index=False)